# AR Aging and AR Trial Balance Recon

# 11/30/25

In [1]:
import pandas as pd
import numpy as np

aged_path = "AR_AgedInvoiceReport.xlsx"
tb_path   = "AR_TrialBalanceDetail.xlsx"



In [2]:
aged_raw = pd.read_excel(aged_path)
tb_raw   = pd.read_excel(tb_path)


/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/envs/anaconda-ai-2024.04-py310/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
aged_raw.head()

,Customer/_x000a_Invoice Date,Unnamed: 1,Invoice _x000a_Number,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Discount_x000a_Amount,_x000a_Balance,_x000a_Current,Unnamed: 12,Unnamed: 13,Unnamed: 14,Days_x000a_Delq,Unnamed: 16
0,NaN,NaT,NaN,NaN,NaN,Invoice,NaN,Discount,NaN,NaN,NaN,NaN,1 Days,31 Days,61 Days,91 Days,NaN
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000003,NaT,Kalispell 3rd Ave Credit Cards,NaN,NaN,NaN,NaN,NaN,Contact:,Margaret Flynn,NaN,Phone:,NaN,NaN,NaN,Credit Limit:,0.0
3,NaN,2025-11-27,W273389-IN,NaN,NaN,2025-12-10 00:00:00,NaN,00:00:00,NaN,0,2654.00,2654,0,0,0,0,0.0
4,NaN,2025-11-28,5993381-PP,NaN,NaN,2025-11-28 00:00:00,NaN,00:00:00,NaN,0,-2297.02,0,-2297.02,0,0,0,0.0


In [4]:
tb_raw.head()

,"CityServiceValcon, LLC (CSV)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Customer/_x000a_Invoice_x000a_Number,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,NaN,__________Dates____________________x000a_Invoi...,NaN,NaN,Invoice Amount,Discount Amount,Invoice Balance,___________Transaction_____________x000a_Type ...,NaT,NaN,Check_x000a_Number/Type,Deposit_x000a_Date
2,0000003 Kalispell 3rd Ave Credit Cards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,5993381-PP,2025-11-28 00:00:00,2025-11-28 00:00:00,00:00:00,-2297.02,0,-2297.02,PRE,2025-11-28,-2297.02,C5993381,2025-11-28 00:00:00
4,W273389-IN,2025-11-27 00:00:00,2025-12-10 00:00:00,00:00:00,2654,0,2654,INV,2025-11-27,2654.00,NaN,NaN


In [5]:
aged = aged_raw.rename(columns={
    "Customer/_x000a_Invoice Date": "raw_customer_or_date",
    "Invoice _x000a_Number": "raw_invoice_or_name",
    "_x000a_Balance": "open_amount",
    "Unnamed: 1": "invoice_date",
})

In [6]:
cust_col = aged["raw_customer_or_date"].astype(str)

# 7-digit DM2 customer numbers (0000003, etc.)
mask_code = cust_col.str.match(r"^\d{7}$")

aged["customer_id"] = np.where(mask_code, cust_col, np.nan)
aged["customer_name"] = np.where(mask_code, aged["raw_invoice_or_name"], np.nan)

aged["customer_id"] = aged["customer_id"].ffill()
aged["customer_name"] = aged["customer_name"].ffill()

In [7]:
aged["invoice_number"] = aged["raw_invoice_or_name"]
aged["open_amount"]    = pd.to_numeric(aged["open_amount"], errors="coerce")
aged["invoice_date"]   = pd.to_datetime(aged["invoice_date"], errors="coerce")

In [8]:
aged_inv = aged[
    aged["invoice_number"].notna() &
    aged["open_amount"].notna()
].copy()

In [9]:
aged_inv.head()
aged_inv[["customer_id", "customer_name", "invoice_number", "open_amount"]].head()

,customer_id,customer_name,invoice_number,open_amount
3,0000003,Kalispell 3rd Ave Credit Cards,W273389-IN,2654.00
4,0000003,Kalispell 3rd Ave Credit Cards,5993381-PP,-2297.02
5,0000003,Kalispell 3rd Ave Credit Cards,W273537-IN,3655.72
9,0000005,North Lewiston Credit Cards,W273391-IN,137.39
10,0000005,North Lewiston Credit Cards,5993502-PP,-114.29


In [10]:
tb = tb_raw.rename(columns={
    "CityServiceValcon, LLC (CSV)": "raw_invoice_or_cust",
    "Unnamed: 1": "invoice_date",
    "Unnamed: 6": "open_amount",
})

In [11]:
cust = tb["raw_invoice_or_cust"].astype(str)

cust_match = cust.str.extract(r"^(?P<customer_id>\d{7})\s+(?P<customer_name>.+)$")

tb["customer_id"] = cust_match["customer_id"]
tb["customer_name"] = cust_match["customer_name"]

tb["customer_id"] = tb["customer_id"].ffill()
tb["customer_name"] = tb["customer_name"].ffill()

In [12]:
tb["invoice_number"] = tb["raw_invoice_or_cust"]
tb["open_amount"]    = pd.to_numeric(tb["open_amount"], errors="coerce")
tb["invoice_date"]   = pd.to_datetime(tb["invoice_date"], errors="coerce")

/tmp/ipykernel_168/2413210351.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tb["invoice_date"]   = pd.to_datetime(tb["invoice_date"], errors="coerce")


In [13]:
tb_inv = tb[tb["open_amount"].notna()].copy()

# Drop "Customer 0000003 Totals:" etc.
tb_inv = tb_inv[~tb_inv["invoice_number"].astype(str).str.startswith("Customer")]

# Drop the customer header rows like "0000003 Kalispell 3rd Ave..."
tb_inv = tb_inv[~tb_inv["invoice_number"].astype(str).str.match(r"^\d{7}\s+")]

# Drop the grand "Report Totals:" line
tb_inv = tb_inv[~tb_inv["invoice_number"].astype(str).str.contains("Report Totals:", na=False)]

In [14]:
tb_inv.head()
tb_inv[["customer_id", "customer_name", "invoice_number", "open_amount"]].head()

,customer_id,customer_name,invoice_number,open_amount
3,0000003,Kalispell 3rd Ave Credit Cards,5993381-PP,-2297.02
4,0000003,Kalispell 3rd Ave Credit Cards,W273389-IN,2654.00
5,0000003,Kalispell 3rd Ave Credit Cards,W273537-IN,3655.72
8,0000005,North Lewiston Credit Cards,5993502-PP,-114.29
9,0000005,North Lewiston Credit Cards,W273391-IN,137.39


In [15]:
aged_total = aged_inv["open_amount"].sum()
tb_total   = tb_inv["open_amount"].sum()

aged_total, tb_total, aged_total - tb_total

(22587985.76, 22616265.62, -28279.859999999404)

In [16]:
inv_recon = aged_inv.merge(
    tb_inv[["customer_id", "invoice_number", "open_amount"]],
    on=["customer_id", "invoice_number"],
    how="outer",
    suffixes=("_aged", "_tb"),
)

inv_recon["open_amount_aged"] = inv_recon["open_amount_aged"].fillna(0)
inv_recon["open_amount_tb"]   = inv_recon["open_amount_tb"].fillna(0)

inv_recon["variance"] = inv_recon["open_amount_aged"] - inv_recon["open_amount_tb"]

In [17]:
inv_issues = inv_recon[inv_recon["variance"].round(2) != 0]

inv_issues.sort_values(["customer_id", "invoice_number"]).head(50)

,raw_customer_or_date,invoice_date,raw_invoice_or_name,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Discount_x000a_Amount,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Days_x000a_Delq,Unnamed: 16,customer_id,customer_name,invoice_number,open_amount_tb,variance
26526,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0005734,NaN,W273640-IN,-550.00,550.00
5892,NaN,2025-11-30,NOV0306-FC,NaN,NaN,2025-11-30 00:00:00,NaN,00:00:00,NaN,0,...,0,0,0,0,0.0,0009230,"Bawa Investments, LLC",NOV0306-FC,0.00,454.65
6725,NaN,2025-11-30,NOV0370-FC,NaN,NaN,2025-11-30 00:00:00,NaN,00:00:00,NaN,0,...,0,0,0,0,0.0,0010546,Parowan Aviation LLC,NOV0370-FC,0.00,468.65
7074,NaN,2025-11-19,0897829-IN,NaN,NaN,2025-11-29 00:00:00,NaN,00:00:00,NaN,0,...,28483.15,0,0,0,1.0,0011133,"Alaska Airlines, Inc.",0897829-IN,28478.15,5.00
7075,NaN,2025-11-19,0897830-IN,NaN,NaN,2025-11-29 00:00:00,NaN,00:00:00,NaN,0,...,28477.45,0,0,0,1.0,0011133,"Alaska Airlines, Inc.",0897830-IN,28472.44,5.01
26527,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0011133,NaN,0901002-IN,31628.34,-31628.34
8172,NaN,2025-11-30,NOV0448-FC,NaN,NaN,2025-11-30 00:00:00,NaN,00:00:00,NaN,0,...,0,0,0,0,0.0,0012466,Rara Corporation,NOV0448-FC,0.00,105.11
10172,NaN,2025-11-30,NOV0589-FC,NaN,NaN,2025-11-30 00:00:00,NaN,00:00:00,NaN,0,...,0,0,0,0,0.0,0015809,"S2S 1994, LLC",NOV0589-FC,0.00,480.87
11745,NaN,2025-11-30,NOV0694-FC,NaN,NaN,2025-11-30 00:00:00,NaN,00:00:00,NaN,0,...,0,0,0,0,0.0,0018146,"Downtown 60%, LLC",NOV0694-FC,0.00,567.80
12036,NaN,2025-11-30,NOV0718-FC,NaN,NaN,2025-11-30 00:00:00,NaN,00:00:00,NaN,0,...,0,0,0,0,0.0,0018596,Geof Gratny,NOV0718-FC,0.00,0.96


In [18]:
aged_cust = (
    aged_inv
    .groupby(["customer_id", "customer_name"], as_index=False)["open_amount"]
    .sum()
    .rename(columns={"open_amount": "aged_open_amount"})
)

tb_cust = (
    tb_inv
    .groupby(["customer_id", "customer_name"], as_index=False)["open_amount"]
    .sum()
    .rename(columns={"open_amount": "tb_open_amount"})
)

cust_recon = aged_cust.merge(
    tb_cust,
    on=["customer_id", "customer_name"],
    how="outer"
)

cust_recon[["aged_open_amount", "tb_open_amount"]] = cust_recon[["aged_open_amount", "tb_open_amount"]].fillna(0)
cust_recon["variance"] = cust_recon["aged_open_amount"] - cust_recon["tb_open_amount"]

cust_issues = cust_recon[cust_recon["variance"].round(2) != 0].sort_values("variance")
cust_issues.head(20)

,customer_id,customer_name,aged_open_amount,tb_open_amount,variance
1752,0011133,"Alaska Airlines, Inc.",2009504.86,2041123.19,-31618.33
7629,0002837,"Bryan Smith Logging, Inc.",0.00,14420.20,-14420.20
3616,0021613,Johnnie L Latham - PPN,-1513.38,0.00,-1513.38
6179,0090564,Johnny Coughlan &,-1274.00,0.00,-1274.00
7632,0013917,Tracee B Jenson-Carroll,0.00,507.27,-507.27
7631,0008923,Johnnie L Egbert,0.00,464.85,-464.85
3037,0018596,Geof Gratny,0.96,0.00,0.96
5051,0060135,Glacier Stone Supply,3743.14,3711.34,31.80
1981,0012466,Rara Corporation,78149.12,78044.01,105.11
5440,0072415,"John C. Berry & Sons, Inc.",15558.39,15328.46,229.93


In [19]:
with pd.ExcelWriter("AR_Recon_Workpapers.xlsx", engine="openpyxl") as writer:
    inv_recon.to_excel(writer, sheet_name="Invoice_Recon_All", index=False)
    inv_issues.to_excel(writer, sheet_name="Invoice_Issues", index=False)
    cust_recon.to_excel(writer, sheet_name="Customer_Recon_All", index=False)
    cust_issues.to_excel(writer, sheet_name="Customer_Issues", index=False)